# Plots for Example Crosson Instance

In [ ]:
using QAOA, Distributions, Interpolations, LinearAlgebra, Arpack
using HDF5, Printf
using PythonPlot
PythonPlot.matplotlib.style.use("./paper.mplstyle")
PythonPlot.rc("axes", prop_cycle=PythonPlot.matplotlib.cycler(color=["#2D5FAA", "#B7293F", "#438E6A", "#F7BD2E", "#F16C37"]))

PATH = "/home/ubuntu/Archives/";
# PLOT_PATH = "/home/ubuntu/Archives/plots/SK_model/paper/";

In [ ]:
using Revise, SpinFluctuations

In [ ]:
Base.show(io::IO, f::Float64) = @printf(io, "%1.4f", f)

In [ ]:
h = [5.0, -1.0, -5.0, 0.0, 0.0, -2.0, -1.0, -3.0, -1.0, -2.0]

J = [0.0 2.0 1.0 1.0 2.0 1.0 1.0 1.0 1.0 1.0;
    2.0 0.0 -1.0 0.0 2.0 1.0 1.0 0.0 0.0 0.0;
    1.0 -1.0 0.0 0.0 -2.0 0.0 0.0 -1.0 0.0 0.0;
    1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0;
    2.0 2.0 -2.0 0.0 0.0 1.0 1.0 1.0 0.0 1.0;
    1.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0;
    1.0 1.0 0.0 0.0 1.0 1.0 0.0 -1.0 0.0 0.0;
    1.0 0.0 -1.0 1.0 1.0 0.0 -1.0 0.0 0.0 0.0;
    1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0]

N = length(h)
mf_problem = Problem(0, h, J);

In [ ]:
h = [-1.0000, 0.0000, 1.0000, -1.0000, -2.0000, -2.0000, 4.0000, -4.0000, -1.0000, -1.0000, 4.0000, -3.0000]

J = [0.0000 0.0000 0.0000 1.0000 0.0000 1.0000 1.0000 0.0000 1.0000 2.0000 2.0000 -1.0000; 0.0000 0.0000 1.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 1.0000; 0.0000 1.0000 0.0000 1.0000 1.0000 -1.0000 2.0000 0.0000 0.0000 0.0000 1.0000 0.0000; 1.0000 0.0000 1.0000 0.0000 0.0000 1.0000 1.0000 0.0000 0.0000 1.0000 0.0000 0.0000; 0.0000 0.0000 1.0000 0.0000 0.0000 0.0000 1.0000 0.0000 0.0000 0.0000 1.0000 1.0000; 1.0000 0.0000 -1.0000 1.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 1.0000 0.0000; 1.0000 0.0000 2.0000 1.0000 1.0000 0.0000 0.0000 1.0000 2.0000 0.0000 1.0000 1.0000; 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 1.0000 0.0000 -1.0000 0.0000 0.0000 0.0000; 1.0000 0.0000 0.0000 0.0000 0.0000 0.0000 2.0000 -1.0000 0.0000 0.0000 1.0000 0.0000; 2.0000 0.0000 0.0000 1.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000; 2.0000 0.0000 1.0000 0.0000 1.0000 1.0000 1.0000 0.0000 1.0000 0.0000 0.0000 1.0000; -1.0000 1.0000 0.0000 0.0000 1.0000 0.0000 1.0000 0.0000 0.0000 0.0000 1.0000 0.0000]

N = length(h)
mf_problem = Problem(0, h, J);

In [ ]:
npts = 8192
coarse_times = range(0, 1, npts + 1);

## Exact gap

In [ ]:
nev = 64
keep_EVs = 5
exact_times = range(0, 1, 33)
eigeninfo = map(s -> (eigs(-SpinFluctuations.hamiltonian(1 - s, s, mf_problem.local_fields, mf_problem.couplings), nev=nev, which=:LM, maxiter=10000)), exact_times)
λs = [vals[1] for vals in eigeninfo]

all_eigvecs = zeros(length(exact_times), 2^N, keep_EVs)
for k in 1:length(exact_times)
    sorting_perm = sortperm(λs[k])
    all_eigvecs[k, :, :] .= eigeninfo[k][2][:, sorting_perm[1:keep_EVs]]
end

In [ ]:
λ = sort(reduce(hcat, λs), dims=1);

In [ ]:
gap = λ[2, :] .- λ[1, :];
mingap = minimum(gap) 
mingap |> println
gap_idx = findfirst(x -> x == mingap, gap) 
gaploc = exact_times[gap_idx] 
gaploc |> println

In [ ]:
figure(figsize=(4, 3))
subplot(111)
for i in 1:size(λ)[1]
    # plot(exact_times[2:end], bogo_spec[i, :], "--C1", lw=1.5, alpha=1, ms=2)
    plot(exact_times, (λ[i, :] .- λ[1, :]), "-k", lw=1, alpha=1, ms=2)    
end
# axvline(gap_interval[1], c="C0")
# axvline(gap_interval[end], c="C0")
plot([], [], "-k", lw=0.75, label="Exact")
plot([], [], "--C1", lw=1.5, label="Bogoliubov")
xlim(0., 1.)
ylim(-0., 2)
xlabel("\$s\$")
ylabel("\$E_\\alpha - E_0\$")
legend(frameon=false)
tight_layout()
display(gcf())
PythonPlot.close();

### Eigenstates

In [ ]:
H_x = SpinFluctuations.hamiltonian(1, 0, mf_problem.local_fields, mf_problem.couplings)
H_z = SpinFluctuations.hamiltonian(0, 1, mf_problem.local_fields, mf_problem.couplings);

In [ ]:
num_eig_vecs = size(all_eigvecs)[3]
eigenstate(n) = [all_eigvecs[k, :, n] for k in 1:length(exact_times)]
eigenstates = [eigenstate(n) for n in 1:num_eig_vecs];
overlap(n, H) = [eigenstates[n][k]' * H * eigenstates[1][k] for k in 1:length(exact_times)]
frac_n(n) = abs.(overlap(n, H_z) .- overlap(n, H_x)) ./ (λ[n, :] .- λ[1, :]);

In [ ]:
gs = [all_eigvecs[k, :, 1] for k in 1:length(exact_times)]
first_ex = [all_eigvecs[k, :, 2] for k in 1:length(exact_times)]
second_ex = [all_eigvecs[k, :, 3] for k in 1:length(exact_times)]

overlap_01_x = overlap(2, H_x)
overlap_01_z = overlap(2, H_z)

overlap_02_x = overlap(3, H_x)
overlap_02_z = overlap(3, H_z);

In [ ]:
figure(figsize=(4, 3))
ax = subplot(111)
plot(exact_times, abs.(overlap_01_z .- overlap_01_x) , label="\$\\alpha = 1\$")
plot(exact_times, abs.(overlap_02_z .- overlap_02_x) , label="\$\\alpha = 2\$", lw=2)
plot(exact_times, ((λ[2, :] .- λ[1, :])), "--C0")
plot(exact_times, ((λ[3, :] .- λ[1, :])), "--C1", lw=2)
xlim(0., 1.)
ylim(0, 1.4)
xlabel("\$s\$")
ylabel("\${|\\langle \\alpha|H_Z-H_X|0\\rangle|}\$")

legend(frameon=false)

tight_layout()
display(gcf())
PythonPlot.close();

## Mean-field trajectories

In [ ]:
T_final = 32768.
tol = 1e-6;

In [ ]:
schedule(t) = t / T_final
sol = evolve_mean_field(mf_problem.local_fields, mf_problem.couplings, T_final, schedule, rtol=tol, atol=tol) 

# get mean-field solution
solution = S -> sign.([S[3, i] for i in 1:size(S)[2]])
mf_sol = solution(sol(T_final)) 

In [ ]:
sol_t = sol.t

sol_u = zeros(length(sol.u), size(sol.u[1])...)
for i in 1:length(sol.u)
    sol_u[i, :, :] .= sol.u[i]
end

size(sol_u)

### Bloch coordinates

In [ ]:
nx_vals = n_vals("x", sol_u)
ny_vals = n_vals("y", sol_u)
nz_vals = n_vals("z", sol_u);

In [ ]:
nx_coarse = n_coarse(nx_vals, sol_t, coarse_times)
ny_coarse = n_coarse(ny_vals, sol_t, coarse_times)
nz_coarse = n_coarse(nz_vals, sol_t, coarse_times);

In [ ]:
S_vals = [transpose(reduce(hcat, [nx_coarse[:, k], ny_coarse[:, k], nz_coarse[:, k]])) |> Matrix for k in 1:npts+1]
magnetizations = reduce(hcat, map(S -> magnetization(S, mf_problem.local_fields, mf_problem.couplings), S_vals));

In [ ]:
areas = Dict()
# dts = [(x[2] - x[1]) / T_final for x in zip(sol_t[1:end-1], sol_t[2:end])]
dts = [(x[2] - x[1]) / T_final for x in zip(coarse_times[1:end-1], coarse_times[2:end])]
for spin_idx in 1:N-1
    # areas[spin_idx] = sum(dts .* nz_vals[spin_idx, 2:end]) |> abs
    areas[spin_idx] = sum(dts .* magnetizations[spin_idx, 2:end]) |> abs
end

top_idxs = [k for (k, v) in sort(areas |> collect, by=x->x[2])]
top_idx = top_idxs[1]

In [ ]:
regular_trajectories = filter!(x -> x != top_idx, collect(1:N-1));

In [ ]:
figure(figsize=(4, 3))

ax = subplot(211)
for spin_nr in regular_trajectories
    plot(coarse_times, nx_coarse[spin_nr, :], "-", c="k")    
end
plot(coarse_times, nx_coarse[top_idx, :], "-", c="C1", label=@sprintf("\$i=%i\$", top_idx))
xlim(0, 1)
ax.set_xticklabels([])
ylim(-1.1, 1.1)
ylabel("\$n_i^x(s)\$")
legend(frameon=false, ncol=2)

ax = subplot(212)
for spin_nr in regular_trajectories
    plot(coarse_times, nz_coarse[spin_nr, :], "-", c="k")    
end
plot(coarse_times, nz_coarse[top_idx, :], "-", c="C1", label=@sprintf("\$i=%i\$", top_idx))
xlim(0, 1)
ylim(-1.1, 1.1)
xlabel("\$s\$")
ylabel("\$n_i^z(s)\$")
# legend(frameon=false, ncol=2)

tight_layout()
display(gcf())
# savefig("../plots/" * @sprintf("mean_field_max2sat_typical_instance_%04i_from_arxiv_2206_06876_N_%i_num_clauses_%i.pdf", idx, N, num_clauses), dpi=256, bbox_inches="tight")
PythonPlot.close();

In [ ]:
figure(figsize=(4, 2))


ax = subplot(111)
for spin_nr in regular_trajectories
    plot(coarse_times, ny_coarse[spin_nr, :], "-", c="k")    
end
plot(coarse_times, ny_coarse[top_idx, :], "-", c="C1", label=@sprintf("\$i=%i\$", top_idx))
xlim(0.0, 1)
# ylim(-1e-3, 1e-3)
xlabel("\$t\$")
ylabel("\$n_i^y(s)\$")
legend(frameon=false)


tight_layout()
display(gcf())
# savefig("../plots/" * @sprintf("mean_field_max2sat_typical_instance_%04i_from_arxiv_2206_06876_N_%i_num_clauses_%i.pdf", idx, N, num_clauses), dpi=256, bbox_inches="tight")
PythonPlot.close();

### Magnetization

In [ ]:
function regularize(x; ε=1e-1)
    pv = x / (x^2 + ε^2)
    abs(x) < ε^2 ? pv : 1/x
end

In [ ]:
figure(figsize=(10, 2.5))


ax = subplot(131)
for spin_nr in regular_trajectories
    plot(coarse_times, magnetizations[spin_nr, :], "-", c="k")
end
plot(coarse_times, magnetizations[top_idx, :], "-", c="C1", label=@sprintf("\$i=%i\$", top_idx))
xlim(0.0, 1)
# ylim(-1000,  1000)
xlabel("\$s\$")
ylabel("\$m_i(s)\$")
legend(frameon=false)

ax = subplot(132)
plot_func = x -> regularize.(x, ε=10e-2)

for spin_nr in regular_trajectories
    # plot(coarse_times, plot_func(abs.(magnetizations[spin_nr, :])), "-", c="k")    
    plot(coarse_times, smoothen(plot_func(abs.(magnetizations[spin_nr, :])), coarse_times, navg=128), "-", c="k")
end
plot(coarse_times, smoothen(plot_func(abs.(magnetizations[top_idx, :])), coarse_times, navg=256), "-", c="C1", label=@sprintf("\$i=%i\$", top_idx))
# plot(coarse_times, plot_func(abs.(magnetizations[top_idx, :])), "-", c="C1", label=@sprintf("\$i=%i\$", top_idx))
axvline(gaploc, ls="--", c="k", alpha=0.5)
xlim(0.0, 1)
ylim(0, 100)
xlabel("\$s\$")
ylabel("\$|m_i(s)|^{-1}\$")
legend(frameon=false)


ax = subplot(133)

for spin_nr in regular_trajectories
    semilogy(coarse_times[2:end], smoothen(map(x -> x[2] - x[1], zip(magnetizations[spin_nr, 1:end-1], magnetizations[spin_nr, 2:end])), coarse_times[2:end], navg=256), "-", c="k")
end
semilogy(coarse_times[2:end], smoothen(map(x -> x[2] - x[1], zip(magnetizations[top_idx, 1:end-1], magnetizations[top_idx, 2:end])), coarse_times[2:end], navg=256) .|> abs, "-", c="C1", label=@sprintf("\$i=%i\$", top_idx))
axvline(gaploc, ls="--", c="k", alpha=0.5)
xlim(0.0, 1)
ylim( )
xlabel("\$s\$")
ylabel("\$|m_i(s)|^{-1}\$")
legend(frameon=false)


tight_layout()
display(gcf())
PythonPlot.close();

### Complex coordinates

In [ ]:
figure(figsize=(4, 3))

ax = subplot(211)
for spin_nr in regular_trajectories
    plot(coarse_times, complex_coordinate(spin_nr, nx_coarse, ny_coarse, nz_coarse) |> real, "-", c="k")    
end
plot(coarse_times, complex_coordinate(top_idx, nx_coarse, ny_coarse, nz_coarse) |> real, "-", c="C1", label=@sprintf("\$i=%i\$", top_idx))
xlim(0, 1)
ylim(-0, 1.5) 
ax.set_xticklabels([])
# xlabel("\$s\$")
ylabel("\$\\mathrm{Re}\\; z_i(s)\$")
legend(frameon=false, ncol=2)

ax = subplot(212)
for spin_nr in regular_trajectories
    plot(coarse_times, (1 .+ abs.(complex_coordinate(spin_nr, nx_coarse, ny_coarse, nz_coarse)).^2).^2, "-", c="k")
end
plot(coarse_times, (1 .+ abs.(complex_coordinate(top_idx, nx_coarse, ny_coarse, nz_coarse)).^2).^2, "-", c="C1", label=@sprintf("\$i=%i\$", top_idx))

xlim(0, 1)
ylim(1, ) 
xlabel("\$s\$")
# ylabel("\$1 + [\\mathrm{Re}\\; z_i(s)]^2\$")
ylabel("\$1 +  |z_i(s)|^2\$")
legend(frameon=false, ncol=2)

tight_layout()
display(gcf())
# savefig("../plots/" * @sprintf("mean_field_max2sat_typical_instance_%04i_from_arxiv_2206_06876_N_%i_num_clauses_%i.pdf", idx, N, num_clauses), dpi=256, bbox_inches="tight")
PythonPlot.close();

### Energies

In [ ]:
# sigma_star = sign.(sol.u[end][3, :])
sigma_star = sign.(sol_u[end, 3, :])
h = mf_problem.local_fields
J = mf_problem.couplings
E = sum([-h[l] * sigma_star[l] for l in 1:N-1]) + sum([-J[i, j] * sigma_star[i] * sigma_star[j] for i in 1:N-1 for j in (i+1):N-1])

In [ ]:
Int.(sigma_star) |> println

In [ ]:
[λ[k, :][end] for k in 1:6] |> println

In [ ]:
for spin_idx in 1:N-1
    # sigma_star_flip = sign.(sol.u[end][3, :])
    sigma_star_flip = sign.(sol_u[end, 3, :])
    sigma_star_flip[spin_idx] = (-1) * sigma_star_flip[spin_idx]
    E_flip = sum([-h[l] * sigma_star_flip[l] for l in 1:N-1]) + sum([-J[i, j] * sigma_star_flip[i] * sigma_star_flip[j] for i in 1:N-1 for j in (i+1):N-1])
    println(spin_idx, ": ", E_flip)
end

In [ ]:
println(top_idxs)

## Statistical Green function

In [ ]:
tol = 1e-8
T_final = 32768.;
# T_final = 65536.;

In [ ]:
lyapunov_parameters = LyapunovParameters(T_final, npts, tol, tol)
mf_sol, stat_GF = statistical_green_function(mf_problem, lyapunov_parameters)

flucs = k -> (real.(1.0im .* diag(stat_GF[k])[1:mf_problem.num_qubits]) .- 1.0) ./ 2;
all_flucs = reduce(hcat, map(flucs, 1:npts+1));

In [ ]:
regular_trajectories = filter!(x -> x != top_idxs[1], collect(1:N-1));
# regular_trajectories = filter!(x -> x != top_idxs[2], regular_trajectories);

scale_factors = [1 .+ abs.(complex_coordinate(i, nx_coarse, ny_coarse, nz_coarse)).^2 for i in 1:N-1]
# mean_scaled_flucs = mean([scale_factors[i].^2 .* smoothen(all_flucs[i, :], coarse_times) for i in regular_trajectories], dims=1)[1];
mean_scaled_flucs = mean([smoothen(all_flucs[i, :], coarse_times) for i in regular_trajectories], dims=1)[1];

In [ ]:
maximum(abs.(overlap_01_z .- overlap_01_x) ./ ((λ[2, :] .- λ[1, :])))

In [ ]:
figure(figsize=(4, 4))
ax = subplot(211)
# semilogy(exact_times, abs.(overlap_01_z .- overlap_01_x) ./ ((λ[2, :] .- λ[1, :])), label="\$\\alpha = 1\$")
# plot(exact_times, abs.(overlap_02_z .- overlap_02_x) ./ ((λ[3, :] .- λ[1, :])), label="\$\\alpha = 2\$", lw=2)
plot(exact_times, frac_n(2) .+ frac_n(3) .+ frac_n(4), "-k", lw=2)
xlim(0., 1.)
# ylim(1e-1, 1e3)
ax.set_xticklabels([])
ylabel("\$\\frac{|\\langle \\alpha|H_Z-H_X|0\\rangle|}{E_\\alpha - E_0}\$")
axvline(gaploc, ls="--", c="k", alpha=0.5)

legend(frameon=false)

reg_func = x -> regularize.(x, ε=9e-2)
# inv_mag = smoothen(reg_func(abs.(magnetizations[top_idx, :])), coarse_times, navg=128)
inv_mag = reg_func(abs.(magnetizations[top_idx, :]))

subplot(212)
scale_factor = 1 .+ abs.(complex_coordinate(top_idx, nx_coarse, ny_coarse, nz_coarse)).^2
plot(coarse_times, smoothen(all_flucs[top_idx, :], coarse_times) .- 0 .* mean_scaled_flucs)
# plot(coarse_times, (smoothen(all_flucs[top_idxs[i], :], coarse_times) .- mean_scaled_flucs) .* inv_mag)

# semilogy(coarse_times, smoothen(scale_factor.^2 .* all_flucs[top_idxs[i], :] .* inv_mag, coarse_times))
# plot(coarse_times, smoothen(scale_factor.^2 .* all_flucs[top_idxs[i], :] .* inv_mag, coarse_times))

axvline(gaploc, ls="--", c="k", alpha=0.5)
xlim(0, 1)
# ylim(1e-1, 1e2)
xlabel("\$s\$")
ylabel("\$ \\langle\\delta \\bar z_i(s) \\delta z_i(s) \\rangle / |m_i(s)|\$")

tight_layout()
display(gcf())
PythonPlot.close();

In [ ]:
figure(figsize=(4, 4))
ax = subplot(211)
semilogy(exact_times, frac_n(2) .+ frac_n(3), "-k", lw=2)
xlim(0., 1.)
ylim(1e-1, 1e3)
ax.set_xticklabels([])
ylabel("\$\\frac{|\\langle \\alpha|H_Z-H_X|0\\rangle|}{E_\\alpha - E_0}\$")
axvline(gaploc, ls="--", c="k", alpha=0.5)

legend(frameon=false)

scaled_flucs(scale_factor, spin_nr) = smoothen(scale_factor.^2 .* all_flucs[spin_nr, :], coarse_times, navg=128)
suscept_denom(spin_nr) = smoothen(map(x -> x[2] - x[1], zip(magnetizations[spin_nr, 1:end-1], magnetizations[spin_nr, 2:end])), coarse_times[2:end], navg=128)

# scaled_flucs(scale_factor, spin_nr) = scale_factor.^2 .* all_flucs[spin_nr, :]
# suscept_denom(spin_nr) = map(x -> x[2] - x[1], zip(magnetizations[spin_nr, 1:end-1], magnetizations[spin_nr, 2:end]))

subplot(212)
scale_factor = 1 .+ abs.(complex_coordinate(top_idx, nx_coarse, ny_coarse, nz_coarse)).^2
plot(coarse_times[2:end], map(x -> x[2] - x[1], zip(scaled_flucs(scale_factor, top_idx)[1:end-1], scaled_flucs(scale_factor, top_idx)[2:end])) ./ suscept_denom(top_idx) .|> abs)
# semilogy(coarse_times[2:end], 
# smoothen(map(x -> x[2] - x[1], zip(scaled_flucs(scale_factor, top_idxs[i])[1:end-1], scaled_flucs(scale_factor, top_idxs[i])[2:end])) ./ suscept_denom(top_idxs[i]),
# coarse_times[2:end], navg=256) .|> abs)

axvline(gaploc, ls="--", c="k", alpha=0.5)
xlim(0, 1)
# ylim(1e-1, 1e2)
xlabel("\$s\$")
ylabel("\$ \\chi_i(s)\$")

tight_layout()
display(gcf())
PythonPlot.close();

In [ ]:
figure(figsize=(8, 3))
normalization = maximum(smoothen(all_flucs[top_idxs[1], :], coarse_times))
normalization = 1.
ylims = (0, maximum(all_flucs ./ normalization))
ylims = (0, )
for i in 1:N÷2
    ax = subplot(2, N÷2, i)
    c = "k"
    if i == top_idx
        c = "C1"
    end
    plot(coarse_times, smoothen(all_flucs[i, :], coarse_times) ./ normalization, label=@sprintf("\$i=%s\$", string(i)), c=c)
    xlim(0., 1.)
    ax.set_xticklabels([])
    ylim(ylims...)
    if i > 1
        ax.set_yticklabels([])
    end
    legend(frameon=false, ncol=2, handlelength=0)
end

ax = subplot(2, N÷2, 1)
ax.set_ylabel("\$F_{ii}(t, t)\$")

for i in N÷2+1:N
    ax = subplot(2, N÷2, i)
    c = "k"
    if i == top_idx
        c = "C1"
    end
    plot(coarse_times, smoothen(all_flucs[i, :], coarse_times) ./ normalization, label=@sprintf("\$i=%s\$", string(i)), c=c)
    xlim(0., 1.)
    ylim(ylims...)
    if i > N÷2+1
        ax.set_yticklabels([])
    end    
    xlabel("\$s\$")    
    legend(frameon=false, ncol=2, handlelength=0)
end

ax = subplot(2, N÷2, N÷2 + 1)
ax.set_ylabel("\$F_{ii}(t, t)\$")

tight_layout()
display(gcf())
# savefig("../plots/" * @sprintf("fluctuations_max2sat_typical_instance_%04i_from_arxiv_2206_06876_N_%i_num_clauses_%i.pdf", idx, N, num_clauses), dpi=256, bbox_inches="tight")
PythonPlot.close();

In [ ]:
figure(figsize=(8, 3))
# normalization = maximum((1 .+ (nx_coarse[top_idxs[1], :] ./ (1 .+ sign(nz_coarse[top_idxs[1], end]) .* nz_coarse[top_idxs[1], :])).^2) .* (smoothen(all_flucs[top_idxs[1], :] .+ 0.0im, coarse_times) .|> sqrt |> real))
normalization = 1.
ylims = (0, 0.2)

for i in 1:N÷2
    ax = subplot(2, N÷2, i)
    c = "k"
    if i == top_idx
        c = "C1"
    end
    scale_factor = 1 .+ abs.(complex_coordinate(i, nx_coarse, ny_coarse, nz_coarse)).^2
    plot(coarse_times, scale_factor.^2 .* smoothen(all_flucs[i, :], coarse_times), label=@sprintf("\$i=%s\$", string(i)), c=c)
    # plot(coarse_times, (scale_factor / normalization) .* (smoothen(all_flucs[i, :] .+ 0.0im, coarse_times) .|> sqrt |> real), label=@sprintf("\$i=%s\$", string(i)), c=c)
    # plot(coarse_times, smoothen(scale_factor .* real(sqrt.(all_flucs[i, :] .+ 0.0im)), coarse_times), label=@sprintf("\$i=%s\$", string(i)), c=c)
    xlim(0., 1.)
    ax.set_xticklabels([])
    ylim(ylims...)
    if i > 1
        ax.set_yticklabels([])
    end
    legend(frameon=false, ncol=2, handlelength=0)
end

ax = subplot(2, N÷2, 1)
ax.set_ylabel("\$F_{ii}(t, t)\$")

for i in N÷2+1:N
    ax = subplot(2, N÷2, i)
    c = "k"
    if i == top_idx
        c = "C1"
    end
    scale_factor = 1 .+ abs.(complex_coordinate(i, nx_coarse, ny_coarse, nz_coarse)).^2
    plot(coarse_times, scale_factor.^2 .* smoothen(all_flucs[i, :], coarse_times), label=@sprintf("\$i=%s\$", string(i)), c=c)
    # plot(coarse_times, (scale_factor / normalization) .*  (smoothen(all_flucs[i, :] .+ 0.0im, coarse_times) .|> sqrt |> real), label=@sprintf("\$i=%s\$", string(i)), c=c)
    # plot(coarse_times, smoothen(scale_factor .* real(sqrt.(all_flucs[i, :] .+ 0.0im)), coarse_times), label=@sprintf("\$i=%s\$", string(i)), c=c)
    xlim(0., 1.)
    ylim(ylims...)
    if i > N÷2+1
        ax.set_yticklabels([])
    end    
    xlabel("\$s\$")    
    legend(frameon=false, ncol=2, handlelength=0)
end

ax = subplot(2, N÷2, N÷2 + 1)
ax.set_ylabel("\$F_{ii}(t, t)\$")

tight_layout()
display(gcf())
# savefig("../plots/" * @sprintf("fluctuations_max2sat_typical_instance_%04i_from_arxiv_2206_06876_N_%i_num_clauses_%i.pdf", idx, N, num_clauses), dpi=256, bbox_inches="tight")
PythonPlot.close();